# Imports

In [1]:
import os
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
import plotly.graph_objs as go

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder

from torchvision.models import googlenet, GoogLeNet_Weights, mobilenet_v3_small, MobileNet_V3_Small_Weights

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device) 
torch.cuda.is_available()

cuda


True

# Load dataset

In [3]:
resize = transforms.Resize(size=(128,128))
hFlip = transforms.RandomHorizontalFlip(p=0.25)
vFlip = transforms.RandomVerticalFlip(p=0.25)
rotate = transforms.RandomRotation(degrees=15)

In [4]:
transform_train = transforms.Compose([resize, hFlip, vFlip, rotate, transforms.ToTensor()])
transform_test = transforms.Compose([resize, transforms.ToTensor()])

In [5]:
trainDataset = ImageFolder(root="./data/Dataset/Train", transform=transform_train)
testDataset = ImageFolder(root="./data/Dataset/Test", transform=transform_test)
validDataset = ImageFolder(root="./data/Dataset/Validation", transform=transform_test)

print(f'[INFO] training dataset contains {len(trainDataset)} samples')
print(f'[INFO] testing dataset contains {len(testDataset)} samples')
print(f'[INFO] validation dataset contains {len(validDataset)} samples')

[INFO] training dataset contains 160000 samples
[INFO] testing dataset contains 20001 samples
[INFO] validation dataset contains 22598 samples


In [6]:
BATCH_SIZE = 64

In [7]:
trainLoader = DataLoader(trainDataset,batch_size=BATCH_SIZE, shuffle= True)
testLoader = DataLoader(testDataset,batch_size=BATCH_SIZE, shuffle= True)
validLoader = DataLoader(validDataset,batch_size=BATCH_SIZE, shuffle= True)

In [8]:
gender_dict = {0: 'Female',
               1: 'Male'}

# Predefine testing method

In [9]:
def test_model(model, dataLoader):
    model.eval()
    
    correct = 0
    total = 0
    
    count = 0
    with torch.no_grad():
        for X, label in dataLoader:
            count += 1
            if(count > 40):
                break
            X = X.to(device=device)
            label = label.to(device=device)
            label = label.reshape(-1,1).float()
        
            odds = model(X).float()
            
            predicted = odds.round()

            total += label.size(0)
            correct += (predicted == label).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Custom Model

In [10]:
class GenderClassification(nn.Module):
    def __init__(self):
        super(GenderClassification, self).__init__()
        self.network = nn.Sequential(
            
            nn.Conv2d(3, 32, kernel_size = 5, stride=3, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(32,64, kernel_size = 5, stride = 3, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2,2),
            
            nn.Flatten(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64, 12),
            nn.ReLU(),
            nn.Linear(12,1),
            nn.Sigmoid()
        )
    
    def forward(self, xb):
        return self.network(xb)

In [11]:
model = GenderClassification().to(device=device)
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)

d:\anaconda3\envs\gender_prediction\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [12]:
NUM_EPOCH = 50

In [13]:
loss_values = []
best_accuracy = -1
for epoch in range(NUM_EPOCH):
    proc_data = 0
    for i in [0,1]:
        if i == 0:
            count = 0
            total_loss = 0
            model.train()
            for X, label in trainLoader:
                count += 1
                if count > 80:
                    break
                optimizer.zero_grad()

                X = X.to(device=device)
                label = label.to(device=device)
                label = label.reshape(-1,1).float()

                odds = model(X).float()
                loss = loss_fn(odds, label)

                loss.backward()
                optimizer.step()

                proc_data += len(X)
                if (proc_data % 2048) == 0:
                    print(f'processed data: {proc_data}/{80 * BATCH_SIZE}')

                total_loss += loss.item()



            avg_loss = total_loss / len(trainLoader)
            loss_values.append(avg_loss)
            print(f'Epoch: {epoch+1}/{NUM_EPOCH}, loss values: {avg_loss}')
        else:
            accuracy = test_model(model, testLoader)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_model = copy.deepcopy(model.state_dict())
            lr_scheduler.step(accuracy)

            print(f'--- Accuracy of the model on the test data: {accuracy:.2f}% ---')

model.load_state_dict(best_model)

processed data: 2048/5120
processed data: 4096/5120
Epoch: 1/50, loss values: 0.01824304951429367
--- Accuracy of the model on the test data: 83.44% ---
processed data: 2048/5120
processed data: 4096/5120
Epoch: 2/50, loss values: 0.012882429349422455
--- Accuracy of the model on the test data: 88.91% ---
processed data: 2048/5120
processed data: 4096/5120
Epoch: 3/50, loss values: 0.010292538130283356
--- Accuracy of the model on the test data: 91.56% ---
processed data: 2048/5120
processed data: 4096/5120
Epoch: 4/50, loss values: 0.009899481999874116
--- Accuracy of the model on the test data: 90.00% ---
processed data: 2048/5120
processed data: 4096/5120
Epoch: 5/50, loss values: 0.008639018890261651
--- Accuracy of the model on the test data: 86.37% ---
processed data: 2048/5120
processed data: 4096/5120
Epoch: 6/50, loss values: 0.00786779597401619
--- Accuracy of the model on the test data: 89.38% ---
processed data: 2048/5120
processed data: 4096/5120
Epoch: 7/50, loss values: 

<All keys matched successfully>

# Playground

In [15]:
# from PIL import Image

In [38]:
# img_1 = Image.open("./photos/...")
# img_2 = Image.open("./photos/...")

# img_1 = transform_test(img_1).to(device)
# img_2 = transform_test(img_2).to(device)

# img_1 = torch.unsqueeze(img_1, 0)
# img_2 = torch.unsqueeze(img_2, 0)

# model.eval()
# with torch.no_grad():
#     output = model(img_2)
#     gender = gender_dict[output.item() >= 0.5]
#     print(f'the gender is: {gender}')


the gender is: Female


# Pretrained models

## setup

In [ ]:
# weights = GoogLeNet_Weights.IMAGENET1K_V1
# model = googlenet(weights=weights).to(device=device)

weights = MobileNet_V3_Small_Weights.IMAGENET1K_V1
model = mobilenet_v3_small(weights=weights).to(device=device)


## Classifier training

In [ ]:
for param in model.parameters():
    param.requires_grad=False

# model.fc = nn.Linear(in_features=model.fc.in_features, out_features=1, bias=True)
model.classifier[3] = nn.Linear(in_features=model.classifier[3].in_features, out_features=1, bias=True).to(device=device)
activation = nn.Sigmoid().to(device=device)


In [ ]:
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.classifier[3].parameters(), lr=1e-3)
# lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)

In [ ]:
NUM_EPOCH = 10

In [ ]:
loss_values = []
for epoch in range(NUM_EPOCH):
    total_loss = 0
    model.train()
    for X, label in trainLoader:
        optimizer.zero_grad()

        X = X.to(device=device)
        label = label.to(device=device)
        label = label.reshape(-1,1).float()
        
        logits = model(X)
        odds = activation(logits).float()
        loss = loss_fn(odds, label)
        
        loss.backward()
        optimizer.step()
    
        total_loss += loss.item()
        
    avg_loss = total_loss / len(trainLoader)
    loss_values.append(avg_loss)
    print(f'Epoch: {epoch+1}/{NUM_EPOCH}, loss values: {avg_loss}')
        
        

In [ ]:
best_model = copy.deepcopy(model.state_dict())

## Finetuning

In [ ]:
for param in model.parameters():
    param.requires_grad=True

In [ ]:
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-2)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)

In [ ]:
model.load_state_dict(best_model)

In [ ]:
best_accuracy = -1
for epoch in range(NUM_EPOCH):
    for i in [0,1]:
        if i == 0:
            total_loss = 0
            model.train()
            for X, label in trainLoader:
                optimizer.zero_grad()

                X = X.to(device=device)
                label = label.to(device=device)
                label = label.reshape(-1,1).float()
                
                logits = model(X)
                odds = activation(logits).float()
                loss = loss_fn(odds, label)
                
                loss.backward()
                optimizer.step()
            
                total_loss += loss.item()
                
                
                
            avg_loss = total_loss / len(trainLoader)
            loss_values.append(avg_loss)
            print(f'Epoch: {epoch+1}/{NUM_EPOCH}, loss values: {avg_loss}')
        else:
            accuracy = test_model(model, testLoader)
            if accuracy > best_accuracy:
                best_model = copy.deepcopy(model.state_dict())
            lr_scheduler.step(accuracy)
            
            print(f'--- Accuracy of the model on the test data: {accuracy:.2f}% ---')
            
model.load_state_dict(best_model)